In [1]:
from google.colab.patches import cv2_imshow
import numpy as np
import cv2

In [2]:
!nvidia-smi

Sat Jul 25 20:00:01 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.51.05    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P8    26W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [4]:
!git clone https://github.com/chandrikadeb7/Face-Mask-Detection.git


Cloning into 'Face-Mask-Detection'...
remote: Enumerating objects: 7, done.
remote: Counting objects: 100% (7/7), done.
remote: Compressing objects: 100% (5/5), done.
remote: Total 3860 (delta 1), reused 4 (delta 1), pack-reused 3853
Receiving objects: 100% (3860/3860), 148.01 MiB | 12.23 MiB/s, done.
Resolving deltas: 100% (43/43), done.
Checking out files: 100% (3864/3864), done.


In [3]:
!git clone https://github.com/prajnasb/observations.git

Cloning into 'observations'...
remote: Enumerating objects: 34, done.
remote: Counting objects: 100% (34/34), done.
remote: Compressing objects: 100% (33/33), done.
remote: Total 1638 (delta 9), reused 0 (delta 0), pack-reused 1604
Receiving objects: 100% (1638/1638), 75.94 MiB | 12.24 MiB/s, done.
Resolving deltas: 100% (20/20), done.


In [5]:
from csv import writer
from imutils import paths
import imutils
import pandas as pd
from sklearn.utils import shuffle
from keras.models import Sequential
from keras.layers import Dense,Activation,Flatten,Dropout
from keras.layers import Conv2D,MaxPooling2D
import sklearn
from sklearn.model_selection import train_test_split
from keras.callbacks import ModelCheckpoint

Using TensorFlow backend.


In [6]:
def resize_to_fit(image, width, height):
    """
    A helper function to resize an image to fit within a given size
    :param image: image to resize
    :param width: desired width in pixels
    :param height: desired height in pixels
    :return: the resized image
    """

    # grab the dimensions of the image, then initialize
    # the padding values
    (h, w) = image.shape[:2]

    # if the width is greater than the height then resize along
    # the width
    if w > h:
        image = imutils.resize(image, width=width)

    # otherwise, the height is greater than the width so resize
    # along the height
    else:
        image = imutils.resize(image, height=height)

    # determine the padding values for the width and height to
    # obtain the target dimensions
    padW = int((width - image.shape[1]) / 2.0)
    padH = int((height - image.shape[0]) / 2.0)

    # pad the image then apply one more resizing to handle any
    # rounding issues
    image = cv2.copyMakeBorder(image, padH, padH, padW, padW,
        cv2.BORDER_REPLICATE)
    image = cv2.resize(image, (width, height))

    # return the pre-processed image
    return image

In [7]:
def append_list_as_row(file_name, list_of_elem):
    # Open file in append mode
    with open(file_name, 'a+', newline='') as write_obj:
        # Create a writer object from csv module
        csv_writer = writer(write_obj)
        # Add contents of list as last row in the csv file
        csv_writer.writerow(list_of_elem)

In [9]:
LETTER_IMAGES_FOLDER0 = "with_mask"
LETTER_IMAGES_FOLDER1 = "without_mask"
LETTER_IMAGES_FOLDER2 = 'wmask'
LETTER_IMAGES_FOLDER3 = 'woutmask'
row_content0 = [1,0]
row_content1 = [0,1]

In [10]:
for image_file in paths.list_images(LETTER_IMAGES_FOLDER0):
    append_list_as_row('Finalmaskdetection.csv', row_content0)

for image_file in paths.list_images(LETTER_IMAGES_FOLDER1):
    append_list_as_row('Finalmaskdetection.csv', row_content1)

for image_file in paths.list_images(LETTER_IMAGES_FOLDER2):
    append_list_as_row('Finalmaskdetection.csv', row_content0)

for image_file in paths.list_images(LETTER_IMAGES_FOLDER3):
    append_list_as_row('Finalmaskdetection.csv', row_content1)  

In [11]:
images = []
for image_file in paths.list_images(LETTER_IMAGES_FOLDER0):
    image = cv2.imread(image_file)
    rgbimage = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    resized=resize_to_fit(rgbimage,120,120)
    images.append(resized/255.0)
for image_file in paths.list_images(LETTER_IMAGES_FOLDER1):
    image = cv2.imread(image_file) 
    rgbimage = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  
    resized=resize_to_fit(rgbimage,120,120)
    images.append(resized/255.0)
for image_file in paths.list_images(LETTER_IMAGES_FOLDER2):
    image = cv2.imread(image_file)
    rgbimage = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    resized=resize_to_fit(rgbimage,120,120)
    images.append(resized/255.0)
for image_file in paths.list_images(LETTER_IMAGES_FOLDER3):
    image = cv2.imread(image_file)
    rgbimage = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    resized=resize_to_fit(rgbimage,120,120)
    images.append(resized/255.0)

In [12]:
images = np.array(images)
images.shape

(5222, 120, 120, 3)

In [14]:
data = pd.read_csv('bestmask.csv')
labels = np.array(data)
labels.shape

(5222, 2)

In [15]:
images, labels = shuffle(images, labels)


In [16]:
X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(images, labels, test_size=0.1)

In [17]:
model=Sequential()

model.add(Conv2D(200,(3,3),input_shape=[120,120,3]))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
#The first CNN layer followed by Relu and MaxPooling layers

model.add(Conv2D(100,(3,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
#The second convolution layer followed by Relu and MaxPooling layers

model.add(Flatten())
model.add(Dropout(0.5))
#Flatten layer to stack the output convolutions from second convolution layer
model.add(Dense(50,activation='relu'))
#Dense layer of 64 neurons
model.add(Dense(2,activation='softmax'))
#The Final layer with two outputs for two categories

model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])


In [18]:
checkpoint = ModelCheckpoint('model-{epoch:03d}.model',monitor='val_loss',verbose=0,save_best_only=True,mode='auto')
history=model.fit(X_train,y_train,epochs=10,callbacks=[checkpoint],validation_data=(X_test, y_test))

Train on 4699 samples, validate on 523 samples
Epoch 1/10
4699/4699 [==============================] - 26s 6ms/step - loss: 0.4936 - accuracy: 0.8108 - val_loss: 0.2778 - val_accuracy: 0.8910
Epoch 2/10
4699/4699 [==============================] - 19s 4ms/step - loss: 0.2383 - accuracy: 0.9157 - val_loss: 0.2393 - val_accuracy: 0.9140
Epoch 3/10
4699/4699 [==============================] - 19s 4ms/step - loss: 0.2012 - accuracy: 0.9268 - val_loss: 0.3558 - val_accuracy: 0.8451
Epoch 4/10
4699/4699 [==============================] - 19s 4ms/step - loss: 0.1870 - accuracy: 0.9270 - val_loss: 0.1980 - val_accuracy: 0.9254
Epoch 5/10
4699/4699 [==============================] - 19s 4ms/step - loss: 0.1452 - accuracy: 0.9464 - val_loss: 0.1997 - val_accuracy: 0.9293
Epoch 6/10
4699/4699 [==============================] - 19s 4ms/step - loss: 0.1347 - accuracy: 0.9468 - val_loss: 0.1892 - val_accuracy: 0.9388
Epoch 7/10
4699/4699 [==============================] - 19s 4ms/step - loss: 0.0990